# Simulation Ensembling Analysis (With Diagnostics)

This notebook runs a full simulation study and **uses the evaluation helpers** to:
- compare losses across ensemble methods
- plot cumulative/rolling losses
- inspect concentration (HHI) dynamics
- summarize what is happening along the way


In [ ]:
import ast
import csv
import math
import sys
import warnings
from pathlib import Path
from typing import Dict, List

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

warnings.filterwarnings("ignore", category=SyntaxWarning)

PROJECT_ROOT = Path.cwd().resolve().parent if Path.cwd().name == "analyses" else Path.cwd().resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

NOTEBOOK_DIR = Path.cwd().resolve() if Path.cwd().name == "analyses" else PROJECT_ROOT / "analyses"
OUT_DIR = NOTEBOOK_DIR / "results"
OUT_DIR.mkdir(parents=True, exist_ok=True)

from src.data import simulator
from src.ensemblers import ensemblers
from src.evaluation import evaluation_helpers as eh

print(f"Project root: {PROJECT_ROOT}")
print(f"Output dir: {OUT_DIR}")


In [ ]:
def mse(y: np.ndarray, yhat: np.ndarray) -> float:
    e = y - yhat
    return float(np.mean(e * e))


def mae(y: np.ndarray, yhat: np.ndarray) -> float:
    return float(np.mean(np.abs(y - yhat)))


def linex(y: np.ndarray, yhat: np.ndarray, a: float = 1.0) -> float:
    e = y - yhat
    return float(np.mean(np.exp(a * e) - a * e - 1.0))


def hhi(weights: np.ndarray) -> float:
    w = np.asarray(weights, dtype=float)
    if w.ndim != 2:
        return math.nan
    valid = np.all(np.isfinite(w), axis=1)
    if not np.any(valid):
        return math.nan
    return float(np.mean(np.sum(w[valid] ** 2, axis=1)))


def helpers_from_source(path: Path) -> List[str]:
    tree = ast.parse(path.read_text(), filename=str(path))
    names: List[str] = []
    for node in tree.body:
        if isinstance(node, (ast.FunctionDef, ast.ClassDef)) and not node.name.startswith("_"):
            names.append(node.name)
    return sorted(names)


def align_for_horizon(
    pi: np.ndarray,
    forecasts_h: np.ndarray,
    s_unc: np.ndarray,
    h: int,
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    y_target = pi[h:]
    F = forecasts_h[:-h]
    s = s_unc[:-h]
    mask = np.isfinite(y_target) & np.isfinite(s) & np.all(np.isfinite(F), axis=1)
    return y_target[mask], F[mask], s[mask]


def evaluate_one_seed(seed: int, horizons: List[int], T: int, window: int) -> List[Dict[str, float]]:
    data, forecasts_by_h, names, s_unc = simulator.make_environment_and_forecasts(
        T=T, horizons=horizons, window=window, seed=seed, include_oracle=False
    )
    pi = data["pi"]
    rows: List[Dict[str, float]] = []

    for h in horizons:
        y, F, s = align_for_horizon(pi, forecasts_by_h[h], s_unc, h)
        individual_mses = [mse(y, F[:, i]) for i in range(F.shape[1])]
        best_idx = int(np.argmin(individual_mses))
        best_individual_mse = float(individual_mses[best_idx])

        configs = {
            "Mean": ensemblers.MeanEnsembler(),
            "Median": ensemblers.MedianEnsembler(),
            "OGDVanilla": ensemblers.OGDVanilla(eta=0.05, loss="squared"),
            "MWUMVanilla": ensemblers.MWUMVanilla(eta=0.3, loss="squared"),
            "OGDBoth": ensemblers.OGDConcentrationBoth(eta=0.05, kappa=0.8, loss="squared"),
            "OGDConcOnly": ensemblers.OGDConcentrationOnly(kappa=0.8, loss="squared"),
            "MWUMBothKL": ensemblers.MWUMBothKL(eta=0.3, kappa=0.8, loss="squared"),
            "MWUMConcOnlyKL": ensemblers.MWUMConcentrationOnlyKL(kappa=0.8, loss="squared"),
        }

        for name, model in configs.items():
            needs_state = name in {"OGDBoth", "OGDConcOnly", "MWUMBothKL", "MWUMConcOnlyKL"}
            result = model.run(F, y, s=s if needs_state else None)
            rows.append({
                "seed": float(seed),
                "horizon": float(h),
                "method": name,
                "n_obs": float(y.size),
                "mse": mse(y, result.yhat),
                "mae": mae(y, result.yhat),
                "linex_a1": linex(y, result.yhat, a=1.0),
                "avg_hhi": hhi(result.weights),
                "best_individual_mse": best_individual_mse,
                "best_individual_idx": float(best_idx),
            })

    return rows


def aggregate(rows: List[Dict[str, float]]) -> pd.DataFrame:
    out: List[Dict[str, float]] = []
    keys = sorted({(r["horizon"], r["method"]) for r in rows})
    for horizon, method in keys:
        grp = [r for r in rows if r["horizon"] == horizon and r["method"] == method]
        mses = np.array([r["mse"] for r in grp], dtype=float)
        maes = np.array([r["mae"] for r in grp], dtype=float)
        linexes = np.array([r["linex_a1"] for r in grp], dtype=float)
        hhis = np.array([r["avg_hhi"] for r in grp], dtype=float)
        bests = np.array([r["best_individual_mse"] for r in grp], dtype=float)

        finite_hhi = hhis[np.isfinite(hhis)]
        avg_hhi_mean = float(np.mean(finite_hhi)) if finite_hhi.size else math.nan

        out.append({
            "horizon": horizon,
            "method": method,
            "mse_mean": float(np.mean(mses)),
            "mse_std": float(np.std(mses)),
            "mae_mean": float(np.mean(maes)),
            "linex_mean": float(np.mean(linexes)),
            "avg_hhi_mean": avg_hhi_mean,
            "avg_excess_mse_vs_best_individual": float(np.mean(mses - bests)),
        })
    df = pd.DataFrame(out).sort_values(["horizon", "mse_mean"]).reset_index(drop=True)
    return df


## 1) What Helpers Do We Have?

Inventory of helper functions/classes in the simulation, ensembling, and evaluation modules.


In [ ]:
SIM_HELPERS = helpers_from_source(PROJECT_ROOT / "src" / "data" / "simulator.py")
ENS_HELPERS = helpers_from_source(PROJECT_ROOT / "src" / "ensemblers" / "ensemblers.py")
EVAL_HELPERS = helpers_from_source(PROJECT_ROOT / "src" / "evaluation" / "evaluation_helpers.py")

display(pd.DataFrame({"src.data.simulator": pd.Series(SIM_HELPERS)}))
display(pd.DataFrame({"src.ensemblers.ensemblers": pd.Series(ENS_HELPERS)}))
display(pd.DataFrame({"src.evaluation.evaluation_helpers": pd.Series(EVAL_HELPERS)}))


## 2) Simulated Macro Background EDA

Before evaluating ensemblers, inspect the simulated macro environment (levels, regimes, uncertainty, and cross-variable relationships).


In [ ]:
SEED_BG = 3
T_BG = 2200

data_bg, _, _, s_unc_bg = simulator.make_environment_and_forecasts(
    T=T_BG, horizons=[1], window=150, seed=SEED_BG, include_oracle=False
)

df_bg = pd.DataFrame({
    "pi": data_bg["pi"],
    "x": data_bg["x"],
    "i": data_bg["i"],
    "u": data_bg["u"],
    "sigma_pi": data_bg["sigma_pi"],
    "regime": data_bg["regime"],
    "state_uncertainty": s_unc_bg,
})

print(f"Background sample size: {len(df_bg)}")
display(df_bg.head())
display(df_bg[["pi","x","i","u","sigma_pi","state_uncertainty"]].describe().T)


In [ ]:
regime_share = (
    df_bg["regime"].value_counts(normalize=True).sort_index().rename("share").to_frame()
)
regime_share.index = [f"regime_{int(i)}" for i in regime_share.index]
display(regime_share)

corr = df_bg[["pi","x","i","u","sigma_pi","state_uncertainty"]].corr()
display(corr)


In [ ]:
# Time-series view with regime path
t = np.arange(len(df_bg))
fig, axes = plt.subplots(6, 1, figsize=(14, 14), sharex=True)

series = ["pi", "x", "i", "u", "sigma_pi", "state_uncertainty"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"]
for ax, sname, c in zip(axes, series, colors):
    ax.plot(t, df_bg[sname].to_numpy(), color=c, linewidth=1.0)
    ax.set_ylabel(sname)
    ax.grid(alpha=0.2)

axr = axes[0].twinx()
axr.step(t, df_bg["regime"].to_numpy(), where="post", color="black", alpha=0.25, linewidth=0.8)
axr.set_ylabel("regime", color="black")
axr.set_yticks([0,1,2])

axes[-1].set_xlabel("time")
fig.suptitle("Simulated Macro Background (with regime path)")
plt.tight_layout()
plt.show()


In [ ]:
# Distribution and regime-conditional views
fig, axes = plt.subplots(2, 2, figsize=(14, 9))

axes[0,0].hist(df_bg["pi"], bins=50, color="#1f77b4", alpha=0.8)
axes[0,0].set_title("Inflation (pi) distribution")

axes[0,1].hist(df_bg["sigma_pi"], bins=50, color="#9467bd", alpha=0.8)
axes[0,1].set_title("Inflation uncertainty (sigma_pi) distribution")

for r, c in zip(sorted(df_bg["regime"].unique()), ["#1f77b4", "#ff7f0e", "#2ca02c"]):
    sub = df_bg[df_bg["regime"] == r]
    axes[1,0].scatter(sub["x"], sub["pi"], s=8, alpha=0.25, color=c, label=f"regime {int(r)}")
axes[1,0].set_title("Phillips-style scatter: pi vs x by regime")
axes[1,0].set_xlabel("x")
axes[1,0].set_ylabel("pi")
axes[1,0].legend()

for r, c in zip(sorted(df_bg["regime"].unique()), ["#1f77b4", "#ff7f0e", "#2ca02c"]):
    sub = df_bg[df_bg["regime"] == r]
    axes[1,1].hist(sub["pi"], bins=35, alpha=0.35, color=c, density=True, label=f"regime {int(r)}")
axes[1,1].set_title("Inflation distribution by regime")
axes[1,1].legend()

for ax in axes.ravel():
    ax.grid(alpha=0.2)

plt.tight_layout()
plt.show()


### EDA Interpretation
- The regime process is persistent (long stretches in each state), not i.i.d.
- `sigma_pi` and `state_uncertainty` vary substantially over time, motivating state-dependent regularization (`lambda_t = kappa * s_t`).
- The `pi`-`x` relationship changes by regime, so static linear pooling can be suboptimal relative to adaptive ensembling.
- Distribution shifts and volatility clustering imply online methods should react differently across tranquil vs turbulent intervals.


## 3) Multi-Seed Aggregate Experiment

Run the full experiment across several random seeds to get stable ranking statistics.


In [ ]:
SEEDS = list(range(10))
HORIZONS = [1, 4, 8]
T = 1400
WINDOW = 150

all_rows: List[Dict[str, float]] = []
for seed in SEEDS:
    all_rows.extend(evaluate_one_seed(seed=seed, horizons=HORIZONS, T=T, window=WINDOW))

agg_df = aggregate(all_rows)
all_df = pd.DataFrame(all_rows)

print(f"Detailed rows: {len(all_df)}")
print(f"Aggregated rows: {len(agg_df)}")
display(agg_df)


### Quick Interpretation

Below we print the best method by MSE per horizon from the aggregate table.


In [ ]:
for h in HORIZONS:
    sub = agg_df[agg_df["horizon"] == float(h)].sort_values("mse_mean")
    top = sub.iloc[0]
    print(
        f"h={h}: best={top['method']} | MSE={top['mse_mean']:.4f}, MAE={top['mae_mean']:.4f}, \
LINEX={top['linex_mean']:.4f}, avg HHI={top['avg_hhi_mean']:.4f}"
    )


## 4) Walkthrough With Evaluation Helpers (Single Scenario)

Now we use `src/evaluation/evaluation_helpers.py` directly to **plot and diagnose what happens over time** for one representative run.


In [ ]:
SEED_DEMO = 0
H_DEMO = 4

data_demo, forecasts_by_h_demo, names_demo, s_unc_demo = simulator.make_environment_and_forecasts(
    T=T, horizons=HORIZONS, window=WINDOW, seed=SEED_DEMO, include_oracle=False
)

y_demo, F_demo, s_demo = align_for_horizon(data_demo["pi"], forecasts_by_h_demo[H_DEMO], s_unc_demo, H_DEMO)

models_demo = {
    "Mean": ensemblers.MeanEnsembler(),
    "Median": ensemblers.MedianEnsembler(),
    "OGDVanilla": ensemblers.OGDVanilla(eta=0.05, loss="squared"),
    "MWUMVanilla": ensemblers.MWUMVanilla(eta=0.3, loss="squared"),
    "OGDBoth": ensemblers.OGDConcentrationBoth(eta=0.05, kappa=0.8, loss="squared"),
    "OGDConcOnly": ensemblers.OGDConcentrationOnly(kappa=0.8, loss="squared"),
    "MWUMBothKL": ensemblers.MWUMBothKL(eta=0.3, kappa=0.8, loss="squared"),
    "MWUMConcOnlyKL": ensemblers.MWUMConcentrationOnlyKL(kappa=0.8, loss="squared"),
}

yhats_demo: Dict[str, np.ndarray] = {}
weights_demo: Dict[str, np.ndarray] = {}
for name, model in models_demo.items():
    needs_state = name in {"OGDBoth", "OGDConcOnly", "MWUMBothKL", "MWUMConcOnlyKL"}
    res = model.run(F_demo, y_demo, s=s_demo if needs_state else None)
    yhats_demo[name] = res.yhat
    weights_demo[name] = res.weights

print(f"Demo setup: seed={SEED_DEMO}, horizon={H_DEMO}, observations={len(y_demo)}, experts={F_demo.shape[1]}")


In [ ]:
# Evaluation helper: summary loss table vs best individual forecaster
table_demo = eh.loss_table(
    y=y_demo,
    F_individual=F_demo,
    yhats=yhats_demo,
    metric="mse",
    include_best_forecaster=True,
    forecaster_names=names_demo,
)
display(table_demo)


### What This Table Says
- The top row is the lowest MSE in this demo scenario.
- The best-individual row is the strongest single forecaster benchmark.
- Ensemble rows below that show whether online weighting beats static/individual baselines.


In [ ]:
# Evaluation helper: cumulative squared loss trajectories
eh.plot_loss_over_time(
    y=y_demo,
    yhats=yhats_demo,
    loss="sq",
    mode="cumulative",
    title=f"Cumulative Squared Loss (seed={SEED_DEMO}, h={H_DEMO})",
)


In [ ]:
# Evaluation helper: rolling loss to see local regime-performance changes
eh.plot_loss_over_time(
    y=y_demo,
    yhats=yhats_demo,
    loss="sq",
    mode="rolling",
    rolling_window=60,
    title=f"Rolling Squared Loss (window=60, seed={SEED_DEMO}, h={H_DEMO})",
)


In [ ]:
# Evaluation helper: concentration dynamics (HHI) from ensemble weights
weights_plot = {k: v for k, v in weights_demo.items() if np.isfinite(v).all()}
eh.plot_hhi_over_time(
    weights_dict=weights_plot,
    title=f"Weight Concentration (HHI) Over Time (seed={SEED_DEMO}, h={H_DEMO})",
)


In [ ]:
# Programmatic narrative from the demo outputs
winner = table_demo.iloc[0]
runner = table_demo.iloc[1]

hhi_avg = {name: eh.hhi_from_weights(W) for name, W in weights_plot.items()}
hhi_mean = {name: float(np.nanmean(v)) for name, v in hhi_avg.items()}
most_concentrated = max(hhi_mean, key=hhi_mean.get)
most_diversified = min(hhi_mean, key=hhi_mean.get)

print("Interpretation:")
print(f"- Best method in this demo (by MSE): {winner['Model']} ({winner['MSE']:.4f})")
print(f"- Runner-up: {runner['Model']} ({runner['MSE']:.4f})")
print(f"- Most concentrated weighting (highest avg HHI): {most_concentrated} ({hhi_mean[most_concentrated]:.4f})")
print(f"- Most diversified weighting (lowest avg HHI): {most_diversified} ({hhi_mean[most_diversified]:.4f})")
print("- In cumulative-loss plots, flatter slope means better ongoing forecast performance.")


## 5) Persist Results

Save detailed/summary metrics and a markdown report for downstream use.


In [ ]:
def write_csv(path: Path, rows: List[Dict[str, float]]) -> None:
    if not rows:
        return
    fieldnames = list(rows[0].keys())
    with path.open("w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        w.writerows(rows)


def write_report(path: Path, sim_helpers: List[str], ens_helpers: List[str], eval_helpers: List[str], agg_df: pd.DataFrame) -> None:
    lines: List[str] = []
    lines.append("# Simulation Ensembling Analysis\n")
    lines.append("## Module Helper Inventory\n")

    lines.append("### `src.data.simulator`")
    lines.extend([f"- `{x}`" for x in sim_helpers])

    lines.append("\n### `src.ensemblers.ensemblers`")
    lines.extend([f"- `{x}`" for x in ens_helpers])

    lines.append("\n### `src.evaluation.evaluation_helpers`")
    lines.extend([f"- `{x}`" for x in eval_helpers])

    lines.append("\n## Aggregated Results")
    for h in sorted(agg_df["horizon"].unique()):
        lines.append(f"\n### Horizon h={int(h)}")
        sub = agg_df[agg_df["horizon"] == h].sort_values("mse_mean")
        for _, r in sub.iterrows():
            lines.append(
                f"- {r['method']}: MSE={r['mse_mean']:.4f} (std {r['mse_std']:.4f}), MAE={r['mae_mean']:.4f}, LINEX={r['linex_mean']:.4f}, avg HHI={r['avg_hhi_mean']:.4f}"
            )

    path.write_text("\n".join(lines) + "\n")


detailed_csv = OUT_DIR / "simulation_ensemble_detailed.csv"
summary_csv = OUT_DIR / "simulation_ensemble_summary.csv"
report_md = OUT_DIR / "simulation_ensemble_report.md"

write_csv(detailed_csv, all_df.to_dict(orient="records"))
write_csv(summary_csv, agg_df.to_dict(orient="records"))
write_report(report_md, SIM_HELPERS, ENS_HELPERS, EVAL_HELPERS, agg_df)

print(f"Wrote: {detailed_csv}")
print(f"Wrote: {summary_csv}")
print(f"Wrote: {report_md}")
